[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_bodies/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

# install tensor2tensor
! pip install -q -U tensor2tensor

# get the needed resources
! curl -O http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.sh
! curl -O http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.py
! curl -O http://josd.github.io/eye/transduction/transduction_bodies/__init__.py
! curl -O http://josd.github.io/eye/transduction/transduction_bodies/test_bodies.observation
! chmod +x observation_prediction_bodies.sh

# clear data and model
% rm -fr /tmp/t2t_data/observation_prediction_bodies
% rm -fr /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4 --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1308  100  1308    0     0   1308      0  0:00:01 --:--:--  0:00:01 12339
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2244  100  2244    0     0   2244      0  0:00:01 --:--:--  0:00:01 36193
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    44  100    44    0     0     44      0  0:00:01 --:--:--  0:00:01   862
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    109      0  0:00:01 --:--:--  0:00:01  2224
  % Total    % Received % Xferd  Average Speed   Tim

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # wind turbine size factor
      size_factor = 

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_l4

USER_DIR=$PWD
DATA_DIR=/tmp/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=/tmp/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train with Adam for 4000 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=4000

# Train with SGD for 1000 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams="optimizer=SGD" \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=5000

# Decode
t2t-decoder \
  --data_dir=$DAT

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
I

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_512.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-21:19:18
INFO:tensorflow:Graph was finalized.
2018-06-17 21:19:18.947386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 21:19:18.947493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] 

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_512.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-21:21:18
INFO:tensorflow:Graph was finalized.
2018-06-17 21:21:18.952399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 21:21:18.952520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 21:21:18.952554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 21:21:18.952586: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 21:21:18.952781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0

INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-21:23:15
INFO:tensorflow:Graph was finalized.
2018-06-17 21:23:16.207246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 21:23:16.207358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 21:23:16.207405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 21:23:16.207456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 21:23:16.207677: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_

INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-21:25:16
INFO:tensorflow:Graph was finalized.
2018-06-17 21:25:17.054302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 21:25:17.054468: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 21:25:17.054505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 21:25:17.054532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 21:25:17.054808: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_

...

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_512.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-22:47:26
INFO:tensorflow:Graph was finalized.
2018-06-17 22:47:27.363794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 22:47:27.363891: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 22:47:27.363950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 22:47:27.363983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 22:47:27.364228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0

INFO:tensorflow:Transforming 'targets' with symbol_modality_873_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-22:49:13
INFO:tensorflow:Graph was finalized.
2018-06-17 22:49:14.542238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 22:49:14.542366: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 22:49:14.542410: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 22:49:14.542441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 22:49:14.542642: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, p

INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-22:51:02
INFO:tensorflow:Graph was finalized.
2018-06-17 22:51:02.796136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 22:51:02.796255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 22:51:02.796312: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 22:51:02.796347: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 22:51:02.796543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_

INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-17-22:52:49
INFO:tensorflow:Graph was finalized.
2018-06-17 22:52:50.292702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 22:52:50.292827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 22:52:50.292867: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 22:52:50.292912: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 22:52:50.293109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_

INFO:tensorflow:Starting evaluation at 2018-06-17-22:54:37
INFO:tensorflow:Graph was finalized.
2018-06-17 22:54:37.693756: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-06-17 22:54:37.693863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-06-17 22:54:37.693916: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-06-17 22:54:37.693960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-06-17 22:54:37.694190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorfl

In [5]:
# See the transductions
# For each observation the top 2 predictions are shown with their respective log probability

! pygmentize -g test_bodies.observation
print("->-")
! pygmentize -g test_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.44	A_PERSON has BMI class O	-5.32
A_TURBINE producing 9533 kW	-0.47	A_TURBINE producing 16682 kW	-1.97


In [6]:
import os

import tensorflow as tf

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization

In [7]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [8]:
# MODEL
CHECKPOINT = os.path.expanduser('/tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4')

In [9]:
# HPARAMS
problem_name = 'observation_prediction_bodies'
data_dir = os.path.expanduser('/tmp/t2t_data/observation_prediction_bodies')
model_name = "transformer"
hparams_set = "transformer_l4"

In [10]:
import observation_prediction_bodies

visualizer = visualization.AttentionVisualizer(hparams_set, model_name, data_dir, problem_name, beam_size=1)

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_512.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_512.top
INFO:tensorflow:Greedy Decoding


In [11]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
input_sentence = "A_TURBINE with size factor 4 and subjected to windspeed 62 km/h"
output_string, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_sentence)
print(output_string)

INFO:tensorflow:Saving checkpoints for 5000 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_l4/model.ckpt.
A_TURBINE producing 9533 kW<EOS>


## Interpreting the Visualizations
- The layers drop down allow you to view the different Transformer layers, 0-indexed of course.
  - Tip: The first layer, last layer and 2nd to last layer are usually the most interpretable.
- The attention dropdown allows you to select different pairs of encoder-decoder attentions:
  - All: Shows all types of attentions together. NOTE: There is no relation between heads of the same color - between the decoder self attention and decoder-encoder attention since they do not share parameters.
  - Input - Input: Shows only the encoder self-attention.
  - Input - Output: Shows the decoder’s attention on the encoder. NOTE: Every decoder layer attends to the final layer of encoder so the visualization will show the attention on the final encoder layer regardless of what layer is selected in the drop down.
  - Output - Output: Shows only the decoder self-attention. NOTE: The visualization might be slightly misleading in the first layer since the text shown is the target of the decoder, the input to the decoder at layer 0 is this text with a GO symbol prepreded.
- The colored squares represent the different attention heads.
  - You can hide or show a given head by clicking on it’s color.
  - Double clicking a color will hide all other colors, double clicking on a color when it’s the only head showing will show all the heads again.
- You can hover over a word to see the individual attention weights for just that position.
  - Hovering over the words on the left will show what that position attended to.
  - Hovering over the words on the right will show what positions attended to it.

In [13]:
call_html()
attention.show(inp_text, out_text, *att_mats)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>